# How well are the most popular roles paid?
- Global Salary Distribution
- Salaries of Top-3 in-Demand Data Roles in Europe
- Salary Gap% for Top-3 Data Roles in US and EU
- Interactive Map - Median Salary for Top-3 Data Roles in EU

In [1]:
import pandas as pd
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go

df = pd.read_csv('/Users/kolesnikevgenia/Documents/Python_Projects/Job_Skills/Raw_Data/df_Final.csv')

In [21]:
#Remove Outliers
# 1. Calculate global IQR
Q1_global = df['salary_month_avg_eur'].quantile(0.25)
Q3_global = df['salary_month_avg_eur'].quantile(0.75)
IQR_global = Q3_global - Q1_global
lower_bound_global = Q1_global - 1.5 * IQR_global
upper_bound_global = Q3_global + 1.5 * IQR_global

# 2. Build list to collect filtered results
filtered_groups = []

# 3. Iterate over groups manually
for (country, title), group in df.groupby(['job_country', 'job_title_short']):
    if len(group) >= 5:
        Q1 = group['salary_month_avg_eur'].quantile(0.25)
        Q3 = group['salary_month_avg_eur'].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
    else:
        lower = lower_bound_global
        upper = upper_bound_global

    filtered = group[
        (group['salary_month_avg_eur'] >= lower) &
        (group['salary_month_avg_eur'] <= upper)
    ]
    filtered_groups.append(filtered)

# 4. Combine all groups back
df_filtered = pd.concat(filtered_groups, ignore_index=True)

# ✅ Final Output
print(f"Original dataset size: {len(df)}")
print(f"Filtered dataset size: {len(df_filtered)}")

Original dataset size: 22003
Filtered dataset size: 21310


### Global Salary Distribution

In [19]:
median_salary = df_filtered.groupby('job_title_short')['salary_month_avg_eur'].median().sort_values()

fig1 = px.bar(
    median_salary,
    x=median_salary.values,
    y=median_salary.index,
    orientation='h',
    title='Global Median Salary by Job Title',
    labels={'x': 'Median Salary, EUR', 'index': 'Job Title'},
    color_discrete_sequence=['#5DADE2'],
    text=median_salary.values
)

fig1.update_traces(
    texttemplate='%{text:.0f}', 
    textposition='inside', 
    insidetextanchor='middle'
    )

fig1.update_layout(
    showlegend=False,
    yaxis_title=''
    )
fig1.show()

### Salaries of Top-3 in-Demand Data Roles in Europe

In [18]:
top_roles = ['Data Analyst', 'Data Scientist', 'Data Engineer']

df_eu_top = df_filtered[
    (df_filtered['job_title_short'].isin(top_roles)) &
    (df_filtered['region_group'] == 'EU')
]

In [20]:
fig2 = px.box(
    df_eu_top,
    x='job_title_short',
    y='salary_month_avg_eur',
    title='Salary Distribution for Top-3 Data Roles in EU',
    labels={
        'salary_month_avg_eur': 'Monthly Salary (EUR)',
        'job_title_short': ''
    }
)

fig2.update_traces(marker_color='#5DADE2', line_color='#5DADE2')

# Add median labels
medians = df_eu_top.groupby('job_title_short')['salary_month_avg_eur'].median().round(0)
for job, med in medians.items():
    fig2.add_annotation(
        x=job,
        y=med,
        text=f"Median: €{med:,.0f}",
        showarrow=False,
        font=dict(size=12),
        yshift=10
    )

fig2.update_layout(showlegend=False)
fig2.show()

### Salary Gap% for Top-3 Data Roles in US and EU

In [22]:
df_top = df[
    (df['region_group'].isin(['EU', 'US'])) &
    (df['job_title_short'].isin(top_roles))
    ]

salary_medians = df_top.groupby(['job_title_short', 'region_group'])['salary_month_avg_eur'].median().unstack()
salary_medians['gap_abs'] = salary_medians['US'] - salary_medians['EU']
salary_medians['gap_pct'] = 100 * (salary_medians['gap_abs'] / salary_medians['EU'])

# Prepare base data
df_base = salary_medians.reset_index()
x_labels = df_base['job_title_short']

# Create the bar chart
fig3 = go.Figure()

# US bars (first in group)
fig3.add_trace(go.Bar(
    x=x_labels,
    y=df_base['US'],
    name='US',
    marker_color='#aec7e8',
    width=0.35,
    text=df_base['US'].round(0),
    textposition='inside',
    insidetextanchor='middle'
))

# EU bars (second in group)
fig3.add_trace(go.Bar(
    x=x_labels,
    y=df_base['EU'],
    name='EU',
    marker_color='#1f77b4',
    width=0.35,
    text=df_base['EU'].round(0),
    textposition='inside',
    insidetextanchor='middle'
))

# Add arrows and % gap labels
for i, row in df_base.iterrows():
    x = row['job_title_short']
    y_us = row['US']
    y_eu = row['EU']
    pct_gap = row['gap_pct']
    y_max = max(y_us, y_eu)

    fig3.add_annotation(
        x=x,
        y=y_max + 800,
        text=f"{pct_gap:.1f}%",
        showarrow=True,
        arrowhead=2,
        arrowwidth=2.5,
        arrowcolor='gray',
        ax=0,
        ay=-40,
        font=dict(size=12, color='black'),
        standoff=6
    )

# Layout settings
fig3.update_layout(
    barmode='group',
    bargap=0.1,  
    title='Salary Gap% for Top-3 Data Roles in US and EU',
    xaxis_title='',
    yaxis_title='Median Salary, EUR',
    height=500,
    legend=dict(title='Region'),
    margin=dict(t=80)
)

fig3.show()

In [7]:
# Manual mapping: job_country → ISO-3
country_iso_map = {
    'Germany': 'DEU', 'France': 'FRA', 'Spain': 'ESP', 'Italy': 'ITA',
    'Netherlands': 'NLD', 'Poland': 'POL', 'Sweden': 'SWE', 'Ireland': 'IRL',
    'Belgium': 'BEL', 'Austria': 'AUT', 'Finland': 'FIN', 'Portugal': 'PRT',
    'Denmark': 'DNK', 'Czech Republic': 'CZE', 'Hungary': 'HUN', 'Romania': 'ROU',
    'Greece': 'GRC', 'Croatia': 'HRV', 'Slovakia': 'SVK', 'Slovenia': 'SVN',
    'Bulgaria': 'BGR', 'Lithuania': 'LTU', 'Latvia': 'LVA', 'Estonia': 'EST',
    'Luxembourg': 'LUX', 'Malta': 'MLT', 'Cyprus': 'CYP'
}

# Group by job_country and job_title_short, then get median salary
df_map = df_eu_top.groupby(['job_country', 'job_title_short'])['salary_month_avg_eur'].median().reset_index()

# Map ISO-3 codes
df_map['iso_alpha'] = df_map['job_country'].map(country_iso_map)

# Drop rows without ISO codes (non-EU or unmapped)
df_map = df_map.dropna(subset=['iso_alpha'])

In [23]:
# 1. Prepare traces for each role (Data Analyst, Scientist, Engineer)
fig4 = go.Figure()

job_titles = df_map['job_title_short'].unique()

for role in job_titles:
    df_role = df_map[df_map['job_title_short'] == role]
    fig4.add_trace(go.Choropleth(
        locations=df_role['iso_alpha'],
        z=df_role['salary_month_avg_eur'],
        text=df_role['job_country'],
        colorscale='Blues',
        colorbar_title='EUR',
        zmin=df_map['salary_month_avg_eur'].min(),
        zmax=df_map['salary_month_avg_eur'].max(),
        visible=(role == job_titles[0]),  # show only first role initially
        name=role,
        locationmode='ISO-3'
    ))

# 2. Side-by-side toggle buttons
buttons = [
    {
        'label': role,
        'method': 'update',
        'args': [
            {'visible': [r == role for r in job_titles]},
            {'title': f'Median Monthly Salary in EU — {role}'}
        ]
    }
    for role in job_titles
]

# 3. Layout with buttons above the plot (not overlapping)
fig4.update_layout(
    title=dict(
        text=f'Median Monthly Salary in EU — {job_titles[0]}',
        y=0.92,  # move title up to make space
        x=0.5,
        xanchor='center'
    ),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth',
        scope='europe'
    ),
    height=600,
    updatemenus=[{
        'type': 'buttons',
        'buttons': buttons,
        'direction': 'right',
        'pad': {'r': 10, 't': 10},
        'x': 0.5,
        'xanchor': 'center',
        'y': 1.05,
        'yanchor': 'top'
    }]
)

fig4.show()

In [9]:
# Filter for Company
ds_sweden = df[
    (df['job_title_short'] == 'Data Scientist') &
    (df['job_country'] == 'Sweden')
]

company_medians = ds_sweden.groupby('company_name')['salary_month_avg_eur'].median().reset_index()
company_medians = company_medians.sort_values(by='salary_month_avg_eur', ascending=False).round(0)

company_medians

,company_name,salary_month_avg_eur
1,Dun & Bradstreet,11560.0
4,Spotify,11560.0
3,H&M Group,10643.0
2,Etraveli Group,6655.0
0,Datatonic,5945.0
